In [1]:
import torch
import numpy as np
import toydiff as tdf

In [2]:
a = tdf.rand((3,3), track_gradient=True)

In [3]:
_a = np.ones((3,3)) * 2
_a[0, 2] = 23
_a[2, 2] = 9

_b = np.ones(3) * 6


ta = torch.Tensor(_a)
ta.requires_grad = True

tb = torch.Tensor(_b)
tb.requires_grad = True

tc = torch.divide(ta, tb)
tc.backward(torch.ones_like(tc))

# --------------------------------
a = tdf.Tensor(_a, track_gradient=True)
b = tdf.Tensor(_b, track_gradient=True)

c = a / b
c.backward()

In [4]:
a.gradient

Tensor([[0.16666667, 0.16666667, 0.16666667],
       [0.16666667, 0.16666667, 0.16666667],
       [0.16666667, 0.16666667, 0.16666667]], dtype=float32, track_gradient=False)

In [5]:
ta.grad.numpy()

array([[0.16666667, 0.16666667, 0.16666667],
       [0.16666667, 0.16666667, 0.16666667],
       [0.16666667, 0.16666667, 0.16666667]], dtype=float32)

In [6]:
b.gradient

Tensor([-0.16666667, -0.16666667, -0.9444445 ], dtype=float32, track_gradient=False)

In [7]:
tb.grad.numpy()

array([-0.16666667, -0.16666667, -0.9444445 ], dtype=float32)

In [8]:
from toydiff.core import OperationRunner

In [9]:
ta.detach().numpy()

array([[ 2.,  2., 23.],
       [ 2.,  2.,  2.],
       [ 2.,  2.,  9.]], dtype=float32)

In [10]:
def generate_input(shape, n_tensors=1):
    def create(shape):
        arr = np.random.rand(*shape)
        tensor = tdf.Tensor(arr, track_gradient=True)
        torch_tensor = torch.Tensor(arr)
        torch_tensor.requires_grad = True
        return tensor, torch_tensor

    tensors = []
    for _ in (0, n_tensors):
        tensors.append(create(shape))
    return tensors

In [11]:
(a, b), (c, d) = generate_input((2,2), 2)

In [12]:
a

Tensor([[0.6424953 , 0.15125677],
       [0.7314589 , 0.31227654]], dtype=float32, track_gradient=True)

In [13]:
np.matmul(np.random.rand(3,2,3), np.random.rand(3,3,3))

array([[[0.52504834, 0.55506119, 1.37928131],
        [0.83672258, 0.64440953, 1.11262993]],

       [[0.31301746, 0.07202308, 0.33111088],
        [0.73947474, 0.11406377, 0.91684541]],

       [[1.36279364, 1.39677136, 1.08815047],
        [0.64468434, 0.65656037, 0.56796719]]])

In [14]:
(t1, t1_torch), (t2, t2_torch) = generate_input((3,))
out = tdf.minimum(t1, t2)
out_torch = torch.minimum(t1_torch, t2_torch)

In [15]:
np.minimum(t1.numpy(), t2.numpy())

array([0.3226355 , 0.13078634, 0.67200273], dtype=float32)

In [16]:
t1

Tensor([0.3226355 , 0.13078634, 0.89679205], dtype=float32, track_gradient=True)

In [17]:
t2

Tensor([0.3644439 , 0.31067124, 0.67200273], dtype=float32, track_gradient=True)

In [18]:
out

Tensor([0.3226355 , 0.13078634, 0.67200273], dtype=float32, backward_fn=<Minimum(BinaryOp).Backward>)

In [19]:
out_torch

tensor([0.3226, 0.1308, 0.6720], grad_fn=<MinimumBackward0>)

In [20]:
a = tdf.rand((3,3), track_gradient=True)
b = tdf.rand((3,3), track_gradient=True)

In [28]:
b = a + 2

In [29]:
b.backward()

In [31]:
b.gradient

Tensor([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]], dtype=float32, track_gradient=False)

In [32]:
_a = torch.rand(3,3)
_a.requires_grad = True

In [33]:
_b = _a + 2

In [34]:
_b.backward(torch.ones_like(_b))

In [35]:
_a.grad

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

In [26]:
e

NameError: name 'e' is not defined

In [ ]:
e.backward()

In [ ]:
a.gradient